# Атрибуция прибыли

Используя разные модели атрибуции прибыли, выясним, какие каналы рекламы приносят больше дохода, где выше эффективность продвижения товара, какие источники лучше для 'холодной' аудитории.

- как зависит ценность сессии от количества сессий у пользователя в линейной модели атрибуции?
- какая сессия для нас ценнее, согласно модели time-decay, которая произошла сутки назад или которая произошла две недели назад?
- в каком случае ценность на одну сессию снижается сильнее, когда у пользователя 5 сессий или когда у него 10 сессий, для модели time-decay?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data.csv', sep = ';', parse_dates = ['date'])
data.head()

,userId,date,trafficSource,cost,value
0,user_10,2020-01-05,telegram / posts,15.75,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   userId         10000 non-null  object        
 1   date           10000 non-null  datetime64[ns]
 2   trafficSource  10000 non-null  object        
 3   cost           10000 non-null  float64       
 4   value          10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 390.8+ KB


In [4]:
data = data.groupby(['userId', 'date', 'trafficSource']).sum().reset_index()

In [5]:
data.sort_values(by=['userId', 'date'], inplace = True)

In [6]:
data.head()

,userId,date,trafficSource,cost,value
0,user_10,2020-01-05,telegram / posts,15.75,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0


In [8]:
# отфильтруем только покупки пользователь
purchases = data.query('value > 0').reset_index()[['userId', 'date']]
purchases

,userId,date
0,user_10,2020-01-05
1,user_1008,2020-01-09
2,user_1011,2020-01-09
3,user_1016,2020-01-03
4,user_1016,2020-01-15
...,...,...
859,user_9975,2020-01-08
860,user_9979,2020-01-08
861,user_9982,2020-01-30
862,user_9985,2020-01-10


In [9]:
# найдем последние покупки по дате
purchases = purchases\
    .groupby('userId')['date']\
    .max()\
    .reset_index()\
    .rename(columns = {'date':'purchaseDate'})

purchases

,userId,purchaseDate
0,user_10,2020-01-05
1,user_1008,2020-01-09
2,user_1011,2020-01-09
3,user_1016,2020-01-15
4,user_1022,2020-01-03
...,...,...
821,user_9975,2020-01-08
822,user_9979,2020-01-08
823,user_9982,2020-01-30
824,user_9985,2020-01-10


In [10]:
# для каждого пользователя мы находим кол-во сессий
data['totalSessions'] = data.groupby('userId')['date'].transform('count') 

# для каждого пользователя мы находим общую сумму покупок
data['totalValue'] = data.groupby('userId')['value'].transform('sum')

# для каждого пользоватлея на ходим номер текущей сессии
data['numberSession'] = data.groupby('userId').cumcount() + 1

# оъединяем с датасетом покупок
session_data = data.merge(purchases, how='left', on='userId')
session_data.head()

,userId,date,trafficSource,cost,value,totalSessions,totalValue
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0


In [20]:
# находим разницу в днях между датой покупки и текущей сессией
session_data['daysToPurchase'] = [(x - y).days if x else 0 
                          for x, y in zip(session_data['purchaseDate'], session_data['date'])]

# по формуле находим значение функции y = 2**(-x/s) - вес каждой сессии
session_data['timeDecayWeight'] = [2 ** (-x / y) 
                                   for x, y in zip(session_data['daysToPurchase'], session_data['numberSession'])]

# находим долю конкретного веса от всех остальных
session_data['timeDecayWeight'] = session_data['timeDecayWeight'] / session_data\
                                .groupby('userId')['timeDecayWeight'].transform('sum')

session_data.head()

,userId,date,trafficSource,cost,value,totalSessions,totalValue,numberSession,purchaseDate,daysToPurchase
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05,0.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT,NaN
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT,NaN
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT,NaN
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT,NaN


In [31]:
# атрибуция по последнему касанию
session_data['lastTouch'] = session_data['value']

# атрибуция по первому касанию 
session_data['firstTouch'] = [x if y == 1 else 0 \
                             for x, y in zip(session_data['totalValue'], session_data['numberSession'])]

# линейная атрибуция прибыли 
session_data['linear'] = session_data['totalValue'] / session_data['totalSessions']

# атрибуция time-decay
session_data['timeDecay'] = session_data['totalValue'] * session_data['timeDecayWeight']

session_data.head()

,userId,date,trafficSource,cost,value,totalSessions,totalValue,numberSession,purchaseDate,daysToPurchase,timeDecayWeight,lastTouch,firstTouch,linear,timeDecay
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05,0.0,1.0,215.0,215.0,215.0,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT,NaN,NaN,0.0,0.0,0.0,NaN
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN


In [32]:
# проверяем расчеты, суммируя распределение прибыли 
session_data[['lastTouch', 'firstTouch', 'linear', 'timeDecay']].sum()

lastTouch     131345.0
firstTouch    131345.0
linear        131345.0
timeDecay     131345.0
dtype: float64

In [33]:
# суммируем прибыль по атрибуциям
totals = session_data.groupby('trafficSource')[['cost', 'lastTouch', 'firstTouch', 'linear', 'timeDecay']].sum()
totals

,cost,lastTouch,firstTouch,linear,timeDecay
trafficSource,,,,,
facebook / video,28208.00,33735.0,38547.5,34870.541667,33996.569902
google / cpc,25137.75,42557.5,41110.0,42677.333333,44574.563738
telegram / posts,35374.50,29330.0,25460.0,27610.208333,28359.125503
vk / display,4498.75,4345.0,4652.5,4116.833333,3520.286662
yandex / cpc,14025.00,21377.5,21575.0,22070.083333,20894.454195


In [42]:
# расcчет ROI

totals['lastTouchROI'] = round(totals['cost'] / totals['lastTouch'] * 100, 2)
totals['firstTouchROI'] = round(totals['cost'] / totals['firstTouch'] * 100, 2)
totals['linearROI'] = round(totals['cost'] / totals['linear'] * 100, 2)
totals['timeDecayROI'] = round(totals['cost'] / totals['timeDecay'] * 100, 2)

In [44]:
totals[['cost', 'lastTouchROI', 'firstTouchROI', 'linearROI', 'timeDecayROI']]

,cost,lastTouchROI,firstTouchROI,linearROI,timeDecayROI
trafficSource,,,,,
facebook / video,28208.00,83.62,73.18,80.89,82.97
google / cpc,25137.75,59.07,61.15,58.90,56.39
telegram / posts,35374.50,120.61,138.94,128.12,124.74
vk / display,4498.75,103.54,96.70,109.28,127.79
yandex / cpc,14025.00,65.61,65.01,63.55,67.12


- *как зависит ценность сессии от количества сессий у пользователя в линейной модели атрибуции?*

чем больше сессий, тем менее ценна каждая из них

- *какая сессия для нас ценнее, согласно модели time-decay, которая произошла сутки назад или которая произошла две недели назад?*

согласно модели time-decay чем ближе сессия к покупке, тем она ценнее, соответственно для нас более ценна сессия, которая произошла сутки назад. 


- *в каком случае ценность на одну сессию снижается сильнее, когда у пользователя 5 сессий или когда у него 10 сессий, для модели time-decay?*

чем меньше сессий, тем сильнее снижается их ценность, значит при 5 сессиях ценность за одну сессию снижается сильнее 